## you/metz w266 project data exploration

In [1]:
import os
import pandas as pd
import numpy as np
import json
import pprint
import re
import nltk
import csv
## nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
!ls -l

total 2048
drwxr-xr-x 1 evely 197609      0 Jul 14 14:37 data
-rw-r--r-- 1 evely 197609  57367 Jul 26 00:54 exploration.ipynb
-rw-r--r-- 1 evely 197609 114043 Jul 25 15:11 exploration_empathy_pairing.ipynb
drwxr-xr-x 1 evely 197609      0 Jul 20 05:39 goemotions
drwxr-xr-x 1 evely 197609      0 Jul 20 19:50 model_trained
-rw-r--r-- 1 evely 197609    781 Jul 21 17:35 README.md
-rw-r--r-- 1 evely 197609   1430 Jul 22 11:04 run_commands
-rwxr-xr-x 1 evely 197609    820 Jul 20 19:50 run_evaluation.sh
-rwxr-xr-x 1 evely 197609    873 Jul 20 19:50 run_training.sh
-rw-r--r-- 1 evely 197609 143730 Jul 20 19:50 Screen Shot 2020-07-18 at 2.45.50 PM.png
-rw-r--r-- 1 evely 197609   1921 Jul 21 17:35 trm_common_commands.md


In [3]:
#data_dir="/home/travismetz/w266_project/data/empatheticdialogues"
data_dir="data"
data_folders=os.listdir(data_dir)
print (data_folders)

['empatheticdialogues', 'goemotions', 'processed', 'dailydialogues', 'MELD', 'other']


In [4]:
for data_folder in data_folders:
    print(data_folder,'\n',os.listdir(os.path.join(data_dir,data_folder)))
    print('-'*30)

empatheticdialogues 
 ['test.csv', 'train.csv', 'valid.csv']
------------------------------
goemotions 
 ['goemotions_1.csv', 'goemotions_2.csv', 'goemotions_3.csv']
------------------------------
processed 
 ['ed_for_goemotions.tsv', 'ed_for_goemotions.tsv.eval_results.txt', 'ed_for_goemotions.tsv.label_predictions.tsv', 'ed_for_goemotions.tsv.predictions.tsv', 'ed_for_goemotions.tsv.tf_record', 'all_dd_data.tsv', 'all_meld_data.tsv', 'all_names.csv', 'dd_for_goemotions.tsv', 'meld_for_goemotions.tsv', 'dd_for_goemotions.tsv.eval_results.txt', 'dd_for_goemotions.tsv.label_predictions.tsv', 'dd_for_goemotions.tsv.predictions.tsv', 'dd_for_goemotions.tsv.tf_record', 'meld_for_goemotions.tsv.eval_results.txt', 'meld_for_goemotions.tsv.label_predictions.tsv', 'meld_for_goemotions.tsv.predictions.tsv', 'meld_for_goemotions.tsv.tf_record', 'ed_train_clean.tsv', 'ed_test_clean.tsv', 'ed_valid_clean.tsv', 'all_ed_data.tsv']
------------------------------
dailydialogues 
 ['dialogues_act.txt',

### empatheticdialogues

In [5]:
data_folder='empatheticdialogues'
file_name='train.csv'
train = pd.read_csv(os.path.join(data_dir, data_folder, file_name), error_bad_lines=False,
                   usecols=[0, 1, 2, 3, 4, 5, 6, 7])
# There are bad lines where columns got out of sequence so need error_bad_lines flag
# EY Note: can solve that by ignoring fields beyond using usecols

In [6]:
train.iloc[0]

conv_id                                               hit:0_conv:1
utterance_idx                                                    1
context                                                sentimental
prompt           I remember going to the fireworks with my best...
speaker_idx                                                      1
utterance        I remember going to see the fireworks with my ...
selfeval                                               5|5|5_2|2|5
tags                                                           NaN
Name: 0, dtype: object

In [7]:
train.shape

(76673, 8)

In [8]:
train.conv_id.unique()

array(['hit:0_conv:1', 'hit:1_conv:2', 'hit:1_conv:3', ...,
       'hit:12423_conv:24846', 'hit:12424_conv:24848',
       'hit:12424_conv:24849'], dtype=object)

In [9]:
train.head()

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,NaN
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,NaN
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,NaN


In [10]:
train.context.value_counts()

surprised       3956
excited         2935
angry           2740
proud           2719
annoyed         2642
sad             2634
afraid          2511
lonely          2503
terrified       2488
grateful        2487
guilty          2463
anxious         2456
disgusted       2447
confident       2440
anticipating    2439
hopeful         2404
furious         2394
impressed       2381
disappointed    2356
nostalgic       2350
joyful          2346
jealous         2329
prepared        2292
content         2215
devastated      2192
embarrassed     2191
sentimental     2073
caring          2054
trusting        2002
ashamed         1976
apprehensive    1822
faithful        1436
Name: context, dtype: int64

In [11]:
def print_conversation(data,conv_id):
    conversation=data[data.conv_id==conv_id]
    print ('context:',conversation.context.iloc[0])
    print('prompt:',conversation.prompt.iloc[0])
    print('selfeval:',conversation.selfeval.iloc[0])
    print('\n')
    for utterance in conversation.utterance:
        print(utterance)

conv_id='hit:0_conv:1'
print_conversation(train,conv_id)

context: sentimental
prompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.
selfeval: 5|5|5_2|2|5


I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.
Was this a friend you were in love with_comma_ or just a best friend?
This was a best friend. I miss her.
Where has she gone?
We no longer talk.
Oh was this something that happened because of an argument?


In [12]:
conversation_list=train.groupby('conv_id').last().reset_index().conv_id
conversation_list

0                hit:0_conv:1
1        hit:10000_conv:20000
2        hit:10000_conv:20001
3        hit:10001_conv:20002
4        hit:10002_conv:20004
                 ...         
17839     hit:9999_conv:19999
17840       hit:999_conv:1998
17841       hit:999_conv:1999
17842         hit:99_conv:198
17843           hit:9_conv:19
Name: conv_id, Length: 17844, dtype: object

In [13]:
for conv_id in conversation_list[0:10]:
    print_conversation(train,conv_id)
    print('\n\n','- '*30)

context: sentimental
prompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.
selfeval: 5|5|5_2|2|5


I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.
Was this a friend you were in love with_comma_ or just a best friend?
This was a best friend. I miss her.
Where has she gone?
We no longer talk.
Oh was this something that happened because of an argument?


 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
context: surprised
prompt: My girlfriend got me a toad today! I was so shocked and happy!
selfeval: 5|5|5_5|5|5


My girlfriend got me a pet toad today!
Do you like toads?
I do! I was so happy when I opened the box and that fat mofo jumped out!
That was nice of your girlfriend_comma_ do you love her?


 - - - - - - - - - - - - - - - - - - - - - - -

### goemotions

In [14]:
data_folder = '../goemotions/data'
file_name = 'test.tsv'
goemotions_test = pd.read_csv(os.path.join(data_dir,data_folder,file_name), sep='\t', header=None)

In [15]:
print(goemotions_test.shape)
goemotions_test.columns = ["utterance", "emotion_labels", "speaker"]

## goemotions format
goemotions_test.head(20)

(5427, 3)


,utterance,emotion_labels,speaker
0,I’m really sorry about your situation :( Altho...,25,eecwqtt
1,It's wonderful because it's awful. At not with.,0,ed5f85d
2,"Kings fan here, good luck to you guys! Will be...",13,een27c3
3,"I didn't know that, thank you for teaching me ...",15,eelgwd1
4,They got bored from haunting earth for thousan...,27,eem5uti
5,Thank you for asking questions and recognizing...,15,ef2nq7i
6,You’re welcome,15,efdbh17
7,100%! Congrats on your job too!,15,ef0ec3b
8,I’m sorry to hear that friend :(. It’s for the...,24,ee8utmi
9,"Girlfriend weak as well, that jump was pathetic.",25,eeni74k


In [16]:
file_name = 'emotions.txt'
emotions = pd.read_csv(os.path.join(data_dir, data_folder, file_name), header=None)
emotions

,0
0,admiration
1,amusement
2,anger
3,annoyance
4,approval
5,caring
6,confusion
7,curiosity
8,desire
9,disappointment


In [17]:
file_name = 'sentiment_dict.json'
with open(os.path.join(data_dir, data_folder, file_name)) as json_data:
    sentiment_map = json.load(json_data)

In [18]:
pprint.pprint(sentiment_map)

{'ambiguous': ['realization', 'surprise', 'curiosity', 'confusion'],
 'negative': ['fear',
              'nervousness',
              'remorse',
              'embarrassment',
              'disappointment',
              'sadness',
              'grief',
              'disgust',
              'anger',
              'annoyance',
              'disapproval'],
 'positive': ['amusement',
              'excitement',
              'joy',
              'love',
              'desire',
              'optimism',
              'caring',
              'pride',
              'admiration',
              'gratitude',
              'relief',
              'approval']}


In [19]:
file_name = 'ekman_mapping.json'
with open(os.path.join(data_dir, data_folder, file_name)) as json_data:
    emotions_group = json.load(json_data)

In [20]:
pprint.pprint(emotions_group)

{'anger': ['anger', 'annoyance', 'disapproval'],
 'disgust': ['disgust'],
 'fear': ['fear', 'nervousness'],
 'joy': ['joy',
         'amusement',
         'approval',
         'excitement',
         'gratitude',
         'love',
         'optimism',
         'relief',
         'pride',
         'admiration',
         'desire',
         'caring'],
 'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
 'surprise': ['surprise', 'realization', 'confusion', 'curiosity']}


### process empatheticdialoges for goemotions classifier

In [21]:
def clean_ed_file(file_name):
    """
    Helper function to help clean the messy ED Files
    """
    
    ## Read in as a text file because of errors and inconsistencies in the file
    text_file = open(os.path.join(data_dir, data_folder, file_name), 'r', encoding='utf-8')
    data_text = text_file.readlines()
    text_file.close()
    
    ## Get each utterance cleaned up
    utterances = list()
    
    for x in range(len(data_text)):
        line = re.split(r',', str(data_text[x]))
        ## ignore all the junk after selfeval
        utterances.append(line[:7])
    
    print("Number of utterances:", len(utterances))
        
    ## Export and export as clean data PD Frame
    columns = utterances[0]
    data_clean = pd.DataFrame(utterances[1:])
    data_clean.columns = columns
    data_clean["tags"] = ""
    
    return data_clean

In [22]:
data_folder = "empatheticdialogues"
ed_train_clean = clean_ed_file('train.csv')

print(ed_train_clean.utterance_idx.value_counts())
ed_train_clean.head()

Number of utterances: 84170
1    19533
2    19531
3    19526
4    19506
5     4370
6      949
7      486
8      268
Name: utterance_idx, dtype: int64


,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,


In [23]:
## Write this out to a file for reference later
ed_train_clean.to_csv ('data/processed/ed_train_clean.tsv', sep="\t", index = False, header=True)

In [24]:
ed_test_clean = clean_ed_file('test.csv')

print(ed_test_clean.utterance_idx.value_counts())
ed_test_clean.head()

Number of utterances: 10974
1    2547
2    2547
3    2546
4    2538
5     552
6     134
7      69
8      40
Name: utterance_idx, dtype: int64


,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:0,1,guilty,I felt guilty when I was driving home one nigh...,0,Yeah about 10 years ago I had a horrifying exp...,2|2|5_5|5|5,
1,hit:0_conv:0,2,guilty,I felt guilty when I was driving home one nigh...,1,Did you suffer any injuries?,2|2|5_5|5|5,
2,hit:0_conv:0,3,guilty,I felt guilty when I was driving home one nigh...,0,No I wasn't hit. It turned out they were drunk...,2|2|5_5|5|5,
3,hit:0_conv:0,4,guilty,I felt guilty when I was driving home one nigh...,1,Why did you feel guilty? People really shouldn...,2|2|5_5|5|5,
4,hit:0_conv:0,5,guilty,I felt guilty when I was driving home one nigh...,0,I don't know I was new to driving and hadn't e...,2|2|5_5|5|5,


In [25]:
## Write this out to a file for reference later
ed_test_clean.to_csv ('data/processed/ed_test_clean.tsv', sep="\t", index = False, header=True)

In [26]:
ed_valid_clean = clean_ed_file('valid.csv')

print(ed_valid_clean.utterance_idx.value_counts())
ed_valid_clean.head()

Number of utterances: 12079
1    2770
2    2769
3    2768
4    2768
5     734
6     162
7      68
8      39
Name: utterance_idx, dtype: int64


,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:3_conv:6,1,terrified,Today_comma_as i was leaving for work in the m...,6,Today_comma_as i was leaving for work in the m...,4|5|5_5|5|5,
1,hit:3_conv:6,2,terrified,Today_comma_as i was leaving for work in the m...,7,Are you fine now?,4|5|5_5|5|5,
2,hit:3_conv:6,3,terrified,Today_comma_as i was leaving for work in the m...,6,Yeah_comma_i'm doing alright now_comma_ but wi...,4|5|5_5|5|5,
3,hit:3_conv:6,4,terrified,Today_comma_as i was leaving for work in the m...,7,Cool :) Is your car damaged a lot?,4|5|5_5|5|5,
4,hit:3_conv:6,5,terrified,Today_comma_as i was leaving for work in the m...,6,The car was badly damaged_comma_i veered outsi...,4|5|5_5|5|5,


In [27]:
## Write this out to a file for reference later
ed_valid_clean.to_csv ('data/processed/ed_valid_clean.tsv', sep="\t", index = False, header=True)

In [60]:
## Combine train and valid datasets, saving test dataset for later
all_ed_data = pd.concat([ed_train_clean, ed_valid_clean])
print(len(all_ed_data))
all_ed_data.head()

96247


,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,


In [61]:
all_ed_data.conv_id.iloc[0]

'hit:0_conv:1'

In [62]:
## Get the Conversation #s
conv_num = list()
for x in range(len(all_ed_data)):
    n = int(re.split(r':', str(all_ed_data.conv_id.iloc[x]))[2])
    conv_num.append(n)

print(len(conv_num))

96247


In [63]:
all_ed_data["conv_num"] = conv_num
all_ed_data.sort_values(by=["conv_num", "utterance_idx"], ascending=True, inplace=True)
all_ed_data.head(10)

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,conv_num
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,,1
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,,1
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,,1
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,,1
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,,1
5,hit:0_conv:1,6,sentimental,I remember going to the fireworks with my best...,0,Oh was this something that happened because of...,5|5|5_2|2|5,,1
6,hit:1_conv:2,1,afraid,i used to scare for darkness,2,it feels like hitting to blank wall when i se...,4|3|4_3|5|5,,2
7,hit:1_conv:2,2,afraid,i used to scare for darkness,3,Oh ya? I don't really see how,4|3|4_3|5|5,,2
8,hit:1_conv:2,3,afraid,i used to scare for darkness,2,dont you feel so.. its a wonder,4|3|4_3|5|5,,2
9,hit:1_conv:2,4,afraid,i used to scare for darkness,3,I do actually hit blank walls a lot of times b...,4|3|4_3|5|5,,2


In [64]:
## Write this out to a file for reference later
all_ed_data.to_csv ('data/processed/all_ed_data.tsv', sep="\t", index = False, header=True)

In [65]:
## Process for goemotions
ed_for_goemotions = pd.DataFrame()
ed_for_goemotions["utterance"] = all_ed_data["utterance"].str.replace("_comma_", ", ", n=- 1, case=None, flags=0, regex=True)
ed_for_goemotions["utterance"] = ed_for_goemotions["utterance"].str.replace("  ", " ", n=- 1, case=None, flags=0, regex=True)
ed_for_goemotions["emotion_labels"] = 0 ## placeholder

## Create a speaker field that has the convo number and the turn of convo
ed_for_goemotions["speaker"] = all_ed_data["conv_num"].map(str) + "_" + all_ed_data["utterance_idx"].map(str)
ed_for_goemotions.head()

,utterance,emotion_labels,speaker
0,I remember going to see the fireworks with my ...,0,1_1
1,"Was this a friend you were in love with, or ju...",0,1_2
2,This was a best friend. I miss her.,0,1_3
3,Where has she gone?,0,1_4
4,We no longer talk.,0,1_5


In [66]:
## Write this out to a file for reference later
ed_for_goemotions.to_csv ('data/processed/ed_for_goemotions.tsv', sep="\t", index = False, header = False)

### process dailydialogues for goemotions classifier

Here are some explanations about the files:

1) dialogues_text.txt: The DailyDialog dataset which contains 11318 transcribed dialogues.

2) dialogues_topic.txt: Each line of dialogues_topic.txt corresponds to the topic of same line in dialogues_text.txt.
                        The representation of the topic number: {1: Ordinary Life, 2: School Life, 3: Culture & Education,
                        4: Attitude & Emotion, 5: Relationship, 6: Tourism , 7: Health, 8: Work, 9: Politics, 10: Finance}

3) dialogues_act.txt: Each line of dialogues_act.txt corresponds to the act of same line in dialogues_text.txt.
                      The representation of the act number: { 1: inform，2: question, 3: directive, 4: commissive }

4) dialogues_emotion.txt: Each line of dialogues_emotion.txt corresponds to the emotion of same line in dialogues_text.txt.
                          The representation of the emotion number: { 0: no emotion, 1: anger, 2: disgust, 3: fear, 4: happiness, 5: sadness, 6: surprise}

In [35]:
data_folder = 'dailydialogues'

file_name = 'dialogues_text.txt'
text_file = open(os.path.join(data_dir, data_folder, file_name), 'r', encoding='utf-8')
dd_text = text_file.readlines()
text_file.close()

print(len(dd_text))
dd_text[:5]

13118


["The kitchen stinks . __eou__ I'll throw out the garbage . __eou__\n",
 'So Dick , how about getting some coffee for tonight ? __eou__ Coffee ? I don ’ t honestly like that kind of stuff . __eou__ Come on , you can at least try a little , besides your cigarette . __eou__ What ’ s wrong with that ? Cigarette is the thing I go crazy for . __eou__ Not for me , Dick . __eou__\n',
 'Are things still going badly with your houseguest ? __eou__ Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . __eou__ Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . __eou__ You ’ re right . Everything is probably going to come 

In [36]:
file_name = 'dialogues_emotion.txt'
text_file = open(os.path.join(data_dir, data_folder, file_name), 'r')
dd_emotion = text_file.readlines()
text_file.close()

len(dd_emotion)

13118

In [37]:
file_name = 'dialogues_act.txt'
text_file = open(os.path.join(data_dir, data_folder, file_name), 'r')
dd_act = text_file.readlines()
text_file.close()

len(dd_act)

13118

In [38]:
file_name = 'dialogues_topic.txt'
text_file = open(os.path.join(data_dir, data_folder, file_name), 'r')
dd_topic = text_file.readlines()
text_file.close()

len(dd_topic)

13118

In [39]:
## Find a way to extract each utterance in the conversations while keeping track of the conversations
convo_lengths = list()
utterances = list()

for x in range(len(dd_emotion)):
    convo_id = x+1
    convo_length = int((len(dd_emotion[x])-1)/2)
    convo_lengths.append(convo_length)
    
    convo = re.split(r'__eou__', dd_text[x])
    emo_tag = re.split(r' ', dd_emotion[x])
    act_tag = re.split(r' ', dd_act[x])
    topic = int(dd_topic[x])
    
    for y in range(convo_length):
        utterances.append([convo_id, topic, y+1, convo[y], act_tag[y], emo_tag[y]])

print("min exchange:", min(convo_lengths))
print("max exchange:", max(convo_lengths))

min exchange: 2
max exchange: 35


In [40]:
dd_data = pd.DataFrame(utterances)
dd_data.columns = ["convo_idx", "topic", "utterance_idx", "utterance", "act_label", "emotion_label"]
dd_data.head(10)

,convo_idx,topic,utterance_idx,utterance,act_label,emotion_label
0,1,1,1,The kitchen stinks .,3,2
1,1,1,2,I'll throw out the garbage .,4,0
2,2,1,1,"So Dick , how about getting some coffee for to...",3,4
3,2,1,2,Coffee ? I don ’ t honestly like that kind of...,4,2
4,2,1,3,"Come on , you can at least try a little , bes...",3,0
5,2,1,4,What ’ s wrong with that ? Cigarette is the t...,1,1
6,2,1,5,"Not for me , Dick .",1,0
7,3,1,1,Are things still going badly with your housegu...,2,0
8,3,1,2,Getting worse . Now he ’ s eating me out of h...,1,1
9,3,1,3,"Leo , I really think you ’ re beating around ...",3,0


#### Handle names, substitute with [NAME] token per goemotions

In [41]:
## find ways to substitute names with [NAME] per goemotions
data_folder = 'other'
file_name = 'baby_names.csv'
baby_names = pd.read_csv(os.path.join(data_dir,data_folder,file_name))
baby_names.head()

,year,name,percent,sex
0,1880,John,0.081541,boy
1,1880,William,0.080511,boy
2,1880,James,0.050057,boy
3,1880,Charles,0.045167,boy
4,1880,George,0.043292,boy


In [42]:
file_name = 'last_names.csv'
last_names = pd.read_csv(os.path.join(data_dir,data_folder,file_name), header=None)
last_names.head()

,0
0,Smith
1,Johnson
2,Williams
3,Brown
4,Jones


In [43]:
## Get a list of English first and last names
eng_names = set(list(baby_names["name"]))
eng_names.update(set(list(last_names[0])))
eng_names.remove("Don") # too often gets confused with "Don't" when processing
eng_names.remove("May") # gets confused when processing
eng_names = pd.DataFrame(eng_names)
eng_names.sort_values(0, ascending=True, inplace=True)

print(len(eng_names))
## print(eng_names)

# Save the file down
eng_names.to_csv ('data/processed/all_names.csv', index = False, header = False)
eng_names = list(eng_names[0])

7367


In [44]:
def process_utterance(utterance, stop_words=eng_names):
    """Helper function to process the utterances into form accepted by goemotions"""
    
    ## Replace names with a single token
    word_tokens = word_tokenize(utterance)
    filtered_sentence = ["[NAME]" if w in stop_words else w for w in word_tokens]
    processed = " ".join(filtered_sentence)
        
    ## Correct spacing and reversing some other minor issues to match with goemotions
    processed = processed.replace(" ,", ",").replace(" .", ".").replace(" ?", "?").replace(" !", "!")
    processed = processed.replace(" ’ ", "'").replace(" ' ", "'")
    processed = processed.replace(" 。", ".")
    processed = processed.replace("", "'")
    
    ## Takes care of full names
    processed = processed.replace("[NAME] [NAME]", "[NAME]")

    return processed

# Test processing with an example
example = 'Joe Hamilton, I really think you’re beating around the bush with this guy.'
process_utterance(example)

"[NAME], I really think you're beating around the bush with this guy."

In [45]:
utterance_list = dd_data.utterance
new_utterance = list()

## Process the utterances similar to goemotions
for utterance in utterance_list:
    new_utterance.append(process_utterance(utterance))

In [46]:
dd_data["utterance"] = new_utterance
dd_data.head(10)

,convo_idx,topic,utterance_idx,utterance,act_label,emotion_label
0,1,1,1,The kitchen stinks.,3,2
1,1,1,2,I 'll throw out the garbage.,4,0
2,2,1,1,"So [NAME], how about getting some coffee for t...",3,4
3,2,1,2,Coffee? I don't honestly like that kind of stuff.,4,2
4,2,1,3,"Come on, you can at least try a little, beside...",3,0
5,2,1,4,What's wrong with that? Cigarette is the thing...,1,1
6,2,1,5,"Not for me, [NAME].",1,0
7,3,1,1,Are things still going badly with your housegu...,2,0
8,3,1,2,Getting worse. Now he's eating me out of house...,1,1
9,3,1,3,"[NAME], I really think you're beating around t...",3,0


In [47]:
# Save the file down for reference later
dd_data.to_csv ('data/processed/all_dd_data.tsv', sep='\t', index = False, header = True)

In [48]:
dd_for_goemotions = pd.DataFrame()
dd_for_goemotions["utterance"] = dd_data["utterance"]
dd_for_goemotions["emotion_labels"] = dd_data["emotion_label"]
dd_for_goemotions["speaker_id"] = dd_data["convo_idx"].map(str) + "_" + dd_data["utterance_idx"].map(str)
dd_for_goemotions.head(10)

,utterance,emotion_labels,speaker_id
0,The kitchen stinks.,2,1_1
1,I 'll throw out the garbage.,0,1_2
2,"So [NAME], how about getting some coffee for t...",4,2_1
3,Coffee? I don't honestly like that kind of stuff.,2,2_2
4,"Come on, you can at least try a little, beside...",0,2_3
5,What's wrong with that? Cigarette is the thing...,1,2_4
6,"Not for me, [NAME].",0,2_5
7,Are things still going badly with your housegu...,0,3_1
8,Getting worse. Now he's eating me out of house...,1,3_2
9,"[NAME], I really think you're beating around t...",0,3_3


In [49]:
# Save the file down
dd_for_goemotions.to_csv ('data/processed/dd_for_goemotions.tsv', sep='\t', index = False, header = False)

### process MELD for goemotions classifier

In [50]:
data_folder = 'MELD'
file_name = 'dev_sent_emo.csv'
meld_dev = pd.read_csv(os.path.join(data_dir, data_folder, file_name))

file_name = 'test_sent_emo.csv'
meld_test = pd.read_csv(os.path.join(data_dir, data_folder, file_name))

all_meld_data = pd.concat([meld_dev, meld_test])
print(all_meld_data.shape)
all_meld_data.head(10)

(3719, 11)


,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,"Oh my God, hes lost it. Hes totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
3,4,Youre a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"
4,5,"Aww, man, now we wont be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505"
5,6,"Now, theres two reasons.",Chandler,neutral,neutral,1,3,4,4,"00:12:38,466","00:12:39,841"
6,7,Hey.,Phoebe,neutral,neutral,1,4,4,4,"0:12:43,012","0:12:44,074"
7,8,Hey!,All,joy,positive,1,5,4,4,"0:12:44,074","0:12:45,189"
8,9,"Ohh, you guys, remember that cute client I tol...",Phoebe,neutral,neutral,1,6,4,4,"00:12:45,347","00:12:50,852"
9,10,Where?!,Rachel,surprise,negative,1,7,4,4,"0:12:52,730","0:12:55,002"


In [51]:
utterance_list = list(all_meld_data["Utterance"])
new_utterance = list()

## Process the utterances similar to goemotions
for utterance in utterance_list:
    new_utterance.append(process_utterance(utterance))

In [52]:
all_meld_data["Utterance"] = new_utterance
all_meld_data.head(10)

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,"Oh my God, he's lost it. He's totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
3,4,You're a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"
4,5,"Aww, man, now we won't be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505"
5,6,"Now, there's two reasons.",Chandler,neutral,neutral,1,3,4,4,"00:12:38,466","00:12:39,841"
6,7,Hey.,Phoebe,neutral,neutral,1,4,4,4,"0:12:43,012","0:12:44,074"
7,8,Hey!,All,joy,positive,1,5,4,4,"0:12:44,074","0:12:45,189"
8,9,"Ohh, you guys, remember that cute client I tol...",Phoebe,neutral,neutral,1,6,4,4,"00:12:45,347","00:12:50,852"
9,10,Where?!,Rachel,surprise,negative,1,7,4,4,"0:12:52,730","0:12:55,002"


In [53]:
## Save down for reference later
all_meld_data.to_csv ('data/processed/all_meld_data.tsv', sep='\t', index = False, header = True)

In [54]:
meld_for_goemotions = pd.DataFrame()
meld_for_goemotions["uterance"] = all_meld_data["Utterance"]
meld_for_goemotions["emotion_labels"] = all_meld_data["Emotion"]
meld_for_goemotions["speaker_id"] = all_meld_data["Speaker"] + "_" + all_meld_data["Dialogue_ID"].map(str) \
+ "_" + all_meld_data["Utterance_ID"].map(str)
meld_for_goemotions.head(10)

,uterance,emotion_labels,speaker_id
0,"Oh my God, he's lost it. He's totally lost it.",sadness,Phoebe_0_0
1,What?,surprise,Monica_0_1
2,"Or! Or, we could go to the bank, close our acc...",neutral,Ross_1_0
3,You're a genius!,joy,Chandler_1_1
4,"Aww, man, now we won't be bank buddies!",sadness,Joey_1_2
5,"Now, there's two reasons.",neutral,Chandler_1_3
6,Hey.,neutral,Phoebe_1_4
7,Hey!,joy,All_1_5
8,"Ohh, you guys, remember that cute client I tol...",neutral,Phoebe_1_6
9,Where?!,surprise,Rachel_1_7


In [55]:
meld_for_goemotions.emotion_labels.value_counts()

neutral     1726
joy          565
anger        498
surprise     431
sadness      319
disgust       90
fear          90
Name: emotion_labels, dtype: int64

In [56]:
## Use for converting emotion_labels to int
emotion_codes = ("neutral", "anger", "disgust", "fear", "joy", "sadness", "surprise")

In [57]:
new_labels = [emotion_codes.index(x) for x in meld_for_goemotions["emotion_labels"]]
meld_for_goemotions["emotion_labels"] = new_labels

In [58]:
meld_for_goemotions.head()

,uterance,emotion_labels,speaker_id
0,"Oh my God, he's lost it. He's totally lost it.",5,Phoebe_0_0
1,What?,6,Monica_0_1
2,"Or! Or, we could go to the bank, close our acc...",0,Ross_1_0
3,You're a genius!,4,Chandler_1_1
4,"Aww, man, now we won't be bank buddies!",5,Joey_1_2


In [59]:
## Save down for goemotions
meld_for_goemotions.to_csv ('data/processed/meld_for_goemotions.tsv', sep='\t', index = False, header = False)